# Climatic data

https://www.renewables.ninja/
Country/Weather data

In [1]:
import pandas as pd
from project.thermal import TEMP_BASE, ORIENTATION_FACTOR

In [3]:
data = pd.read_csv('ninja_weather_country_FR_merra-2_population_weighted.csv', skiprows=[0, 1], index_col=[0], parse_dates=[0])

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 350640 entries, 1980-01-01 00:00:00 to 2019-12-31 23:00:00
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   precipitation       350640 non-null  float64
 1   temperature         350640 non-null  float64
 2   irradiance_surface  350640 non-null  float64
 3   irradiance_toa      350640 non-null  float64
 4   snowfall            350640 non-null  float64
 5   snow_mass           350640 non-null  float64
 6   cloud_cover         350640 non-null  float64
 7   air_density         350640 non-null  float64
dtypes: float64(8)
memory usage: 24.1 MB


In [5]:
variables = ['temperature', 'irradiance_surface']

## Calculation

In [6]:
d_hs_yr, temp_hs_yr, irr_hs_yr = dict(), dict(), dict()
d_hs_month, temp_hs_month, irr_hs_month = dict(), dict(), dict()
d_hs_day, temp_hs_day, irr_hs_day = dict(), dict(), dict()

for year in data.index.year.unique():

    d = data.loc[data.index.year == year, variables]
    d = d.groupby([d.index.month, d.index.day]).agg({'temperature': 'mean', 'irradiance_surface': 'sum'})
    d.index.names = ['month', 'day']

    hs = d['temperature'] < TEMP_BASE

    # yearly data
    d_hs_yr.update({year: hs.sum()})
    temp_hs_yr.update({year: (hs * d['temperature']).sum() / d_hs_yr[year]})
    irr_hs_yr.update({year:  (hs * d['irradiance_surface']).sum() / 1000})

    # monthly data
    d_hs_month.update({year: hs.groupby('month').sum()})
    temp_hs_month.update({year: (hs * d['temperature']).groupby('month').sum() / d_hs_month[year]})
    irr_hs_month.update({year: (hs * d['irradiance_surface']).groupby('month').sum() / 1000})

    # dayly data
    d_hs_day.update({year: hs.groupby(['month', 'day']).sum()})
    temp_hs_day.update({year: (hs * d['temperature']).groupby(['month', 'day']).sum() / d_hs_month[year]})
    irr_hs_day.update({year: (hs * d['irradiance_surface']).groupby(['month', 'day']).sum() / 1000})

result_year = pd.concat((pd.Series(d_hs_yr), pd.Series(temp_hs_yr), pd.Series(irr_hs_yr)), axis=1, keys=['d_hs', 'temp_hs', 'irr_hs (kWh/m2)'])
result_month = pd.concat((pd.DataFrame(d_hs_month).rename_axis('year', axis=1).stack(), pd.DataFrame(temp_hs_month).rename_axis('year', axis=1).stack(), pd.DataFrame(irr_hs_month).rename_axis('year', axis=1).stack()), axis=1, keys=['d_hs', 'temp_hs', 'irr_hs (kWh/m2)'])
result_day = pd.concat((pd.DataFrame(d_hs_day).rename_axis('year', axis=1).stack(), pd.DataFrame(temp_hs_day).rename_axis('year', axis=1).stack(), pd.DataFrame(irr_hs_day).rename_axis('year', axis=1).stack()), axis=1, keys=['d_hs', 'temp_hs', 'irr_hs (kWh/m2)'])

In [7]:
var = 'irr_hs (kWh/m2)'
for key, factor in ORIENTATION_FACTOR.items():
    result_year['{} {}'.format(var, key)] = result_year[var] * factor
    result_month['{} {}'.format(var, key)] = result_month[var] * factor
    result_day['{} {}'.format(var, key)] = result_day[var] * factor

In [20]:
# exporting data
export_resirf = result_year[['d_hs', 'temp_hs', 'irr_hs (kWh/m2) Mean']]
export_resirf.columns = ['TEMP_EXT', 'DAYS_HEATING_SEASON', 'SOLAR_RADIATION']
export_resirf.to_csv('output/climatic_data.csv')

In [17]:
result_year[['d_hs', 'temp_hs', 'irr_hs (kWh/m2) Mean']].set

,d_hs,temp_hs,irr_hs (kWh/m2) Mean
1980,222,5.516114,445.150005
1981,202,5.472071,368.429845
1982,216,6.298965,424.871092
1983,219,5.981987,431.223989
1984,228,6.217092,473.712847
1985,207,4.276391,407.562637
1986,205,4.910095,380.507676
1987,207,4.846042,405.583265
1988,195,6.379806,360.396069
1989,191,6.396070,373.657812


In [10]:
result_month

d_hs   temp_hs  irr_hs (kWh/m2)  irr_hs (kWh/m2) South  \
month year                                                           
1     1980    31  1.143880        39.232040              47.078448   
      1981    31  2.292302        42.152691              50.583229   
      1982    31  3.562700        37.523820              45.028584   
      1983    31  4.660469        39.602662              47.523194   
      1984    31  3.434234        33.456170              40.147404   
...          ...       ...              ...                    ...   
12    2015    31  7.824118        36.849436              44.219323   
      2016    31  4.078780        40.049761              48.059713   
      2017    31  4.093655        30.156358              36.187629   
      2018    31  5.896011        31.823411              38.188093   
      2019    31  6.039973        34.179842              41.015811   

            irr_hs (kWh/m2) West  irr_hs (kWh/m2) Est  irr_hs (kWh/m2) North  \
month year                                                                     
1     1980             29.424030            29.424030              12.554253   
      1981             31.614518            31.614518              13.488861   
      1982             28.142865            28.142865              12.007622   
      1983             29.701996            29.701996              12.672852   
      1984             25.092128            25.092128              10.705974   
...                          ...                  ...                    ...   
12    2015             27.637077            27.637077              11.791820   
      2016             30.037321            30.037321              12.815923   
      2017             22.617268            22.617268               9.650034   
      2018             23.867558            23.867558              10.183492   
      2019             25.634882            25.634882              10.937550   

            irr_hs (kWh/m2) Mean  
month year                        
1     1980             29.620190  
      1981             31.825282  
      1982             28.330484  
      1983             29.900010  
      1984             25.259409  
...                          ...  
12    2015             27.821324  
      2016             30.237569  
      2017             22.768050  
      2018             24.026675  
      2019             25.805781  

[480 rows x 8 columns]

In [11]:
pd.DataFrame(d_hs_day).rename_axis('Year', axis=1).stack()

month  day  Year
1      1    1980    1.0
            1981    1.0
            1982    1.0
            1983    1.0
            1984    1.0
                   ... 
12     31   2015    1.0
            2016    1.0
            2017    1.0
            2018    1.0
            2019    1.0
Length: 14610, dtype: float64

In [12]:
result_month

d_hs   temp_hs  irr_hs (kWh/m2)  irr_hs (kWh/m2) South  \
month year                                                           
1     1980    31  1.143880        39.232040              47.078448   
      1981    31  2.292302        42.152691              50.583229   
      1982    31  3.562700        37.523820              45.028584   
      1983    31  4.660469        39.602662              47.523194   
      1984    31  3.434234        33.456170              40.147404   
...          ...       ...              ...                    ...   
12    2015    31  7.824118        36.849436              44.219323   
      2016    31  4.078780        40.049761              48.059713   
      2017    31  4.093655        30.156358              36.187629   
      2018    31  5.896011        31.823411              38.188093   
      2019    31  6.039973        34.179842              41.015811   

            irr_hs (kWh/m2) West  irr_hs (kWh/m2) Est  irr_hs (kWh/m2) North  \
month year                                                                     
1     1980             29.424030            29.424030              12.554253   
      1981             31.614518            31.614518              13.488861   
      1982             28.142865            28.142865              12.007622   
      1983             29.701996            29.701996              12.672852   
      1984             25.092128            25.092128              10.705974   
...                          ...                  ...                    ...   
12    2015             27.637077            27.637077              11.791820   
      2016             30.037321            30.037321              12.815923   
      2017             22.617268            22.617268               9.650034   
      2018             23.867558            23.867558              10.183492   
      2019             25.634882            25.634882              10.937550   

            irr_hs (kWh/m2) Mean  
month year                        
1     1980             29.620190  
      1981             31.825282  
      1982             28.330484  
      1983             29.900010  
      1984             25.259409  
...                          ...  
12    2015             27.821324  
      2016             30.237569  
      2017             22.768050  
      2018             24.026675  
      2019             25.805781  

[480 rows x 8 columns]

In [13]:
result_month

d_hs   temp_hs  irr_hs (kWh/m2)  irr_hs (kWh/m2) South  \
month year                                                           
1     1980    31  1.143880        39.232040              47.078448   
      1981    31  2.292302        42.152691              50.583229   
      1982    31  3.562700        37.523820              45.028584   
      1983    31  4.660469        39.602662              47.523194   
      1984    31  3.434234        33.456170              40.147404   
...          ...       ...              ...                    ...   
12    2015    31  7.824118        36.849436              44.219323   
      2016    31  4.078780        40.049761              48.059713   
      2017    31  4.093655        30.156358              36.187629   
      2018    31  5.896011        31.823411              38.188093   
      2019    31  6.039973        34.179842              41.015811   

            irr_hs (kWh/m2) West  irr_hs (kWh/m2) Est  irr_hs (kWh/m2) North  \
month year                                                                     
1     1980             29.424030            29.424030              12.554253   
      1981             31.614518            31.614518              13.488861   
      1982             28.142865            28.142865              12.007622   
      1983             29.701996            29.701996              12.672852   
      1984             25.092128            25.092128              10.705974   
...                          ...                  ...                    ...   
12    2015             27.637077            27.637077              11.791820   
      2016             30.037321            30.037321              12.815923   
      2017             22.617268            22.617268               9.650034   
      2018             23.867558            23.867558              10.183492   
      2019             25.634882            25.634882              10.937550   

            irr_hs (kWh/m2) Mean  
month year                        
1     1980             29.620190  
      1981             31.825282  
      1982             28.330484  
      1983             29.900010  
      1984             25.259409  
...                          ...  
12    2015             27.821324  
      2016             30.237569  
      2017             22.768050  
      2018             24.026675  
      2019             25.805781  

[480 rows x 8 columns]

In [14]:
result

NameError: name 'result' is not defined

In [ ]:
data.loc[data.index.year == year, variables]

In [ ]:
result

In [ ]:
d

In [ ]:
duration

In [ ]:
data.index